In [2]:
import torchvision
import torch
from heatmap import HeatmapModel, generate_heatmap

model = HeatmapModel(1, [128, 128])
model.load_state_dict(torch.load('steering_model.pth'))

device = torch.device('cuda')
model = model.to(device).eval().half()

In [3]:
import torchvision
import torch
from heatmap import HeatmapModel, generate_heatmap

speed_model = torchvision.models.resnet18(pretrained=True)
speed_model.fc = torch.nn.Linear(512, 2)
speed_model.load_state_dict(torch.load('throttle_model.pth'))

device = torch.device('cuda')
speed_model = speed_model.to(device).eval().half()

In [4]:
import tensorrt as trt
from torch2trt import torch2trt, tensorrt_converter

In [3]:
#from torch2trt import torch2trt, tensorrt_converter, 

data = torch.zeros((1, 3, 224, 224)).cuda().half()

model_trt = torch2trt(model, [data], fp16_mode=True, max_workspace_size=1<<25)

NameError: name 'torch2trt' is not defined

In [5]:

data = torch.zeros((1, 3, 224, 224)).cuda().half()

speed_model_trt = torch2trt(speed_model, [data], fp16_mode=True, max_workspace_size=1<<25)

We can save the model by calling the following.  This way we can re-use the optimized model without having to run the optimization procedure again.

In [ ]:
torch.save(model_trt.state_dict(), 'steering_model_trt.pth')

In [7]:
torch.save(speed_model_trt.state_dict(), 'throttle_model_trt.pth')

We can save the optimized model by loading it into a TRTModule as follows.

In [1]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('steering_model_trt.pth'))

In [2]:
import torch
from torch2trt import TRTModule

speed_model_trt = TRTModule()
speed_model_trt.load_state_dict(torch.load('throttle_model_trt.pth'))

Next, let's create the racecar instance which we'll use to control the robot. 

In [3]:
from jetracer.nvidia_racecar import NvidiaRacecar

car = NvidiaRacecar(
    steering_gain=1.0,
    steering_offset=0.0,
    throttle_gain=-1.0,
    throttle_offset=0.0
)

And some sliders to tune parameters used for steering the robot.

In [4]:
from IPython.display import display
import ipywidgets

steering_gain = ipywidgets.FloatSlider(min=0.0, max=3.0, value=0.5, step=0.001, description='steering gain')
steering_offset = ipywidgets.FloatSlider(min=-0.5, max=0.5, value=0.0, step=0.001, description='steering offset')
throttle_gain = ipywidgets.FloatSlider(min=0.0, max=2.0, value=1.0, step=0.001, description='throttle gain')
throttle_offset = ipywidgets.FloatSlider(min=-0.5, max=0.5, value=0.0, step=0.001, description='throttle offset')

display(
    steering_gain,
    steering_offset,
    throttle_gain,
    throttle_offset
)

FloatSlider(value=0.5, description='steering gain', max=3.0, step=0.001)

FloatSlider(value=0.0, description='steering offset', max=0.5, min=-0.5, step=0.001)

FloatSlider(value=1.0, description='throttle gain', max=2.0, step=0.001)

FloatSlider(value=0.0, description='throttle offset', max=0.5, min=-0.5, step=0.001)

In [5]:
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, value=0.0, step=0.001)
throttle_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, value=0.0, step=0.001)

display(steering_slider, throttle_slider)

FloatSlider(value=0.0, max=1.0, min=-1.0, step=0.001)

FloatSlider(value=0.0, max=1.0, step=0.001)

Call the following cell to initialize the camera.

In [6]:
image_widget = ipywidgets.Image(format='jpeg')

display(image_widget)

Image(value=b'', format='jpeg')

In [8]:
from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg

camera = CSICamera(width=224, height=224, capture_width=1280, capture_height=720, capture_fps=60)

camera.running = True

In [9]:
import trt_pose.plugins

def get_best_peak(heatmap, thresh=0.1):
    counts, peaks = trt_pose.plugins.find_peaks(heatmap, thresh, 5, 50)
    refined_peaks = trt_pose.plugins.refine_peaks(counts, peaks, heatmap, 5)
    vals = []
    if counts[0][0] == 0:
        return None
    for i in range(counts[0][0]):
        vals.append(float(heatmap[0, 0, int(peaks[0, 0, i, 0]), int(peaks[0, 0, i, 1])]))
    idx = np.argmax(vals)
    val = vals[idx]
    peak = refined_peaks[0, 0, idx]
    return peak

In [23]:
from utils import preprocess
import cv2
import numpy as np

# log = []

def execute(change):
    image = change['new']
    data = preprocess(image)
    heatmap = model_trt(data).cpu().float()#.numpy()
    
    peak = get_best_peak(heatmap)
    if peak is not None:
        yint = int(peak[0] * 224)
        xint = int(peak[1] * 224)
        y = float(1.0 - peak[0])
        x = 2.0 * (float(peak[1]) - 0.5)
        car.steering = float(np.clip(x * abs(x) * steering_gain.value + steering_offset.value, -1.0, 1.0))
        cv2.circle(image, (xint, yint), 3, (0, 255, 0), 3)
        steering_slider.value = car.steering
        
    speedxy = speed_model_trt(data).cpu().float()
    speedy = 0.5 - 0.5 * float(speedxy[0, 1])
    car.throttle = speedy * throttle_gain.value + throttle_offset.value
    throttle_slider.value = car.throttle
    
    image_widget.value = bgr8_to_jpeg(image)

We'll manually call the execution function once, which may take some time the first time the neural network is called.

We recommend disabling autonomous mode for this first execution.

In [24]:
execute({'new': camera.value})

Now, assuming the cell above executed without issue, call the following cell to attach the execute function to new camera values.

In [25]:
camera.observe(execute, names='value')

In [22]:
camera.unobserve_all()